In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import django
django.setup()

In [2]:
from performance_benchmark.models import Benchmark
from django.utils import timezone
from uuid import uuid4
from random import randint, randrange, choice
from datetime import datetime, timedelta, timezone
from string import ascii_lowercase, ascii_letters
from django.db.models import Count, Q, F

In [3]:
def get_random_date():
    step = timedelta(days=1)
    start = datetime(2020, 1, 1)
    end = datetime.now()
    random_date = start + randrange((end - start) // step + 1) * step
    return random_date

In [4]:
def get_random_email():
    return "".join(choice(ascii_lowercase) for _ in range(randint(8,15))) + '@joey.com'
def get_random_string():
    return "".join(choice(ascii_letters) for _ in range(randint(8,15)))

In [114]:
benchmark_objects = []
for i in range(25):
    db_random_iteration = randint(5, 25)
    client_id = randint(1, 4)
    databook_id = uuid4()
    for j in range(db_random_iteration):
        knowledge_begin_date = get_random_date()
        ds_random_iteration = randint(5, 25)
        datasheet_id = uuid4()
        for k in range(ds_random_iteration):
            benchmar_obj = Benchmark(
                knowledge_begin_date=knowledge_begin_date,
                client_id=client_id,
                databook_id=databook_id,
                datasheet_id=datasheet_id,
                data={
                    "amount": randint(-500, 501),
                    "email": get_random_email(),
                    "name": get_random_string(),
                    "address": get_random_string(),
                },
            )
            benchmark_objects.append(benchmar_obj)
# Benchmark.objects.bulk_create(benchmark_objects, batch_size=None, ignore_conflicts=False)

558 ms ± 36.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
import time
import math
def calculate_time(func):
    def inner1(*args, **kwargs):
        begin = time.time()         
        func(*args, **kwargs)
        end = time.time()
        print("Total time taken in seconds : ", func.__name__, end - begin)
    return inner1
 

## Difference between only, defer, values and values_list
### Objective is to fetch distinct databook_ids from the entire table.

### Only -  Takes only that column value, but allows you to query other things as well(Does extra query)

In [6]:
start_time = time.time()

benchmark_objects = Benchmark.objects.only('databook_id')

unique_databook_ids = set()
for each_object in benchmark_objects:
    unique_databook_ids.add(each_object.databook_id)
    
print(benchmark_objects[0].data)

print('Time Taken: '+ str(time.time() - start_time) + ' seconds')

{'amount': 10}
Time Taken: 23.780166149139404 seconds


### Defer -  Takes all Columns except the defered, but allows you to query that thing as well(Does extra query)

In [7]:
start_time = time.time()

benchmark_objects = Benchmark.objects.defer('data')

unique_databook_ids = set()
for each_object in benchmark_objects:
    unique_databook_ids.add(each_object.databook_id)
    
print(benchmark_objects[0].data)

print('Time Taken: '+ str(time.time() - start_time) + ' seconds')

{'amount': 10}
Time Taken: 40.66952180862427 seconds


### values - but applying it without selecting any columns.

In [8]:
start_time = time.time()

benchmark_objects = Benchmark.objects.all().values()

unique_databook_ids = set()
for each_object in benchmark_objects:
    unique_databook_ids.add(each_object['databook_id'])
    

print('Time Taken: '+ str(time.time() - start_time) + ' seconds')

Time Taken: 38.55430889129639 seconds


### values - And selecting only the column we need

In [9]:
start_time = time.time()

benchmark_objects = Benchmark.objects.values('databook_id')

unique_databook_ids = set()
for each_object in benchmark_objects:
    unique_databook_ids.add(each_object['databook_id'])
    
try:
    print(benchmark_objects[0]['data'])
except:
    print('Exception: Data is not accessible here')
                                                    
print('Time Taken: '+ str(time.time() - start_time) + ' seconds')

Exception: Data is not accessible here
Time Taken: 10.518049955368042 seconds


### values_list - Fetching the single column with flat and converting it to set

In [10]:
start_time = time.time()

unique_databook_ids = set(Benchmark.objects.values_list('databook_id',flat=True))

print('Time Taken: '+ str(time.time() - start_time) + ' seconds')

Time Taken: 6.727375030517578 seconds


### values_list - Getting distinct value from the Query

In [11]:
start_time = time.time()

unique_databook_ids = set(Benchmark.objects.values_list('databook_id',flat=True).distinct())


print('Time Taken: '+ str(time.time() - start_time) + ' seconds')

Time Taken: 0.9716320037841797 seconds


## .count vs len()

### Using Python's Vanilla len() function

In [12]:
start_time = time.time()

unique_databook_ids_length = len(set(Benchmark.objects.values_list('databook_id',flat=True).distinct()))
print(unique_databook_ids_length)

print('Time Taken: '+ str(time.time() - start_time) + ' seconds')

32370
Time Taken: 0.953031063079834 seconds


### Using Django's .count() in Query

In [13]:
start_time = time.time()

unique_databook_ids_length = Benchmark.objects.values_list('databook_id',flat=True).distinct().count()
print(unique_databook_ids_length)

print('Time Taken: '+ str(time.time() - start_time) + ' seconds')

32370
Time Taken: 0.8960278034210205 seconds


## Objective: check whether databook_id and datasheet_id pair exist

### Fetching object and checking len > 0

In [14]:
start_time = time.time()

db_ds_objects = Benchmark.objects.filter(databook_id='61722a62-fe71-44df-86a5-477bcdfbd91c', datasheet_id='d065e9da-804c-4272-88a0-6e467a50573e')

if len(db_ds_objects) > 0:
    print('Yes, this object exist')


print('Time Taken: '+ str(time.time() - start_time) + ' seconds')

Yes, this object exist
Time Taken: 0.29444074630737305 seconds


### Checking with .count()

In [15]:
start_time = time.time()

db_ds_objects = Benchmark.objects.filter(databook_id='61722a62-fe71-44df-86a5-477bcdfbd91c', datasheet_id='d065e9da-804c-4272-88a0-6e467a50573e').count()

if db_ds_objects > 0:
    print('Yes, this object exist')


print('Time Taken: '+ str(time.time() - start_time) + ' seconds')

Yes, this object exist
Time Taken: 0.29352283477783203 seconds


### Using .exist() function

In [16]:
start_time = time.time()

db_ds_objects = Benchmark.objects.filter(databook_id='61722a62-fe71-44df-86a5-477bcdfbd91c', datasheet_id='d065e9da-804c-4272-88a0-6e467a50573e').exists()

if db_ds_objects:
    print('Yes, this object exist')


print('Time Taken: '+ str(time.time() - start_time) + ' seconds')

Yes, this object exist
Time Taken: 0.07770609855651855 seconds


### Objective: Count records with Databook_id = X or Datasheet_Id = Y but not the records where both conditions satisfies simultaneously.

### Double Query and Filter

In [17]:
start_time = time.time()

databook_condition_objs = Benchmark.objects.filter(
    databook_id="61722a62-fe71-44df-86a5-477bcdfbd91c"
)
datasheet_condition_objs = Benchmark.objects.filter(
    datasheet_id="5044b179-ddf1-4305-8473-dae5e5e0ff91"
)
combined_objects_before_check = []

for each_object in databook_condition_objs:
    combined_objects_before_check.append(each_object)

for each_object in datasheet_condition_objs:
    combined_objects_before_check.append(each_object)

combined_objects_after_check = []

for each_object in combined_objects_before_check:
    if not (
        each_object.databook_id == "61722a62-fe71-44df-86a5-477bcdfbd91c"
        and each_object.datasheet_id == "5044b179-ddf1-4305-8473-dae5e5e0ff91"
    ):
        combined_objects_after_check.append(each_object)

print(len(combined_objects_after_check))

print("Time Taken: " + str(time.time() - start_time) + " seconds")


35
Time Taken: 0.5853922367095947 seconds


### Using Q objects

In [18]:

start_time = time.time()

databook_condition = Q(databook_id="61722a62-fe71-44df-86a5-477bcdfbd91c")
datasheet_condition = Q(datasheet_id="5044b179-ddf1-4305-8473-dae5e5e0ff91")
both_condition = Q(databook_id="61722a62-fe71-44df-86a5-477bcdfbd91c") & Q(
    datasheet_id="5044b179-ddf1-4305-8473-dae5e5e0ff91"
)

combined_objects_after_check = Benchmark.objects.filter(
    (databook_condition | datasheet_condition) & ~both_condition
).count()
print(combined_objects_after_check)


print("Time Taken: " + str(time.time() - start_time) + " seconds")


35
Time Taken: 0.3725588321685791 seconds


### Note: Q Objects precedence should be higher than other filters, other wise the query is invalid

In [19]:
Benchmark.objects.filter(
    Q(databook_id="61722a62-fe71-44df-86a5-477bcdfbd91c"),
    datasheet_id="d065e9da-804c-4272-88a0-6e467a50573e",
)
## this is simply AND condition and VALID QUERY

<QuerySet [<Benchmark: Benchmark object (523153)>, <Benchmark: Benchmark object (523154)>]>

In [20]:
Benchmark.objects.filter(
    datasheet_id="d065e9da-804c-4272-88a0-6e467a50573e",
    Q(databook_id="61722a62-fe71-44df-86a5-477bcdfbd91c"),
)  
### ERROR

SyntaxError: positional argument follows keyword argument (713771487.py, line 4)

### Objective: Updating Objects depending on the data in the table. Set Knowledge_end_date = knowledge_begin_date + 10 days.

### Two things,
### 1. Iterate the objects, get knowledge_begin_date, add 10 days to it and call .save() method
### 2. Iterate the objects, get knowledge_begin_date, add 10 days to it, store it in list and call bulk_update method


### Iterating and calling .save

In [21]:
start_time = time.time()

objects = Benchmark.objects.filter(datasheet_id="d065e9da-804c-4272-88a0-6e467a50573e")
for each_object in objects:
    each_object.knowledge_end_date = each_object.knowledge_begin_date + timedelta(days=10)
    each_object.save()

print("Time Taken: " + str(time.time() - start_time) + " seconds")

Time Taken: 0.2963268756866455 seconds


### Using F()

In [22]:
start_time = time.time()

Benchmark.objects.filter(
    datasheet_id="d065e9da-804c-4272-88a0-6e467a50573e",
).update(knowledge_end_date = F('knowledge_begin_date') + timedelta(days=10))

print("Time Taken: " + str(time.time() - start_time) + " seconds")

Time Taken: 0.29288220405578613 seconds


#### Things to Note:
  #### 1.  Avoiding Race Condition in F()
  #### 2. Single Query Instead of Querying and Saving every single time.

## Use .iterator if the number of objects to fetch large

### Fetching all the objects

In [29]:
start_time = time.time()

databook_id = set()
benchmark_objects = Benchmark.objects.all().values('databook_id')
for obj in benchmark_objects:
    databook_ids.add(obj['databook_id'])


print(len(databook_ids))
print("Time Taken: " + str(time.time() - start_time) + " seconds")

266220
Time Taken: 9.415884017944336 seconds


### Fetching chunked records through Iterator

In [24]:
start_time = time.time()

databook_ids = set()
benchmark_objects = Benchmark.objects.all().values('databook_id').iterator(chunk_size=2000)
for obj in benchmark_objects:
    databook_ids.add(obj['databook_id'])

print(len(databook_ids))
print("Time Taken: " + str(time.time() - start_time) + " seconds")

32370
Time Taken: 6.472586154937744 seconds


### Use litreal syntax for declarling list, set, tuple and dictionary

In [30]:
start_time = time.time()

for i in range(10**7):
    l=list() 

print("Time Taken: " + str(time.time() - start_time) + " seconds")

Time Taken: 1.2021896839141846 seconds


In [31]:
start_time = time.time()

for i in range(10**7):
    l=[]


print("Time Taken: " + str(time.time() - start_time) + " seconds")

Time Taken: 0.6270980834960938 seconds


### Check truthness of object without length

In [32]:
large_list = [i for i in range(10**2)]

if large_list: pass ## Prefer this
    
if len(large_list) > 0: pass ## over this.
